In [1]:
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
import xgboost as xgb
from pyod.models.iforest import IForest

In [2]:
df = pd.read_csv('data/fe_data_with_compliance_thresholds.csv')

In [3]:
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
df.head()

,transaction_reference,merchant_bears_cost,amount,is_fraud,created_hod,phone_number_diff,unique_phone_numbers,diff_number_txns,total_transactions,proportion_different_numbers,average_phone_number_difference,shared_phone_number_flag,txn_count_per_customer,txn_count_per_customer_per_hour,max_txns_in_an_hour_per_customer,txn_count_per_customer_per_day,phone_txn_amount_suspicious,rolling_txn_count_per_customer,txn_spike_flag,avg_txn_amount_per_customer,unusual_txn_flag,rolling_mean_amount,amount_spike_flag,duplicate_txn_flag,odd_hour_flag,is_Exness,is_Nexasoft,is_Headway,is_Blenet,is_Hongkong,is_Bitolo,is_Onus,is_Astropay,is_Spoynt,is_Gateexpress,account_risk_level_above_average_risk,account_risk_level_high_risk,payment_status_expired,payment_status_failed,payment_status_success,payment_channel_api,payment_channel_modal,mobile_network_provider_Airtel,mobile_network_provider_MTN,mobile_network_provider_Vodafone,is_success,is_insufficient_funds,is_charge_attempt_failed,is_wallet_not_active,is_invalid_recipient_wallet,is_invalid_PIN,created_dow_Friday,created_dow_Monday,created_dow_Saturday,created_dow_Sunday,created_dow_Thursday,created_dow_Tuesday,created_dow_Wednesday,confidence_score,temp_fraud
0,KPY-CA-r0hEfAhffbv0,1,153.0,0,2,0.0,1,0,1,0.0,0.0,1,1,1,1,1,0,1.0,0,153.0,0,153.0,0,0,1,0,1,0,0,0,0,0,0,0,0,1,0,0,0,1,1,0,0,1,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0.090909,0
1,KPY-CA-BVbXPVkbZ10N,1,1000.0,0,14,0.0,1,0,3,0.0,0.0,1,3,1,1,1,0,1.0,1,445.0,0,1000.0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,1,1,0,0,1,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0.090909,0
2,KPY-CA-BD6zJ3HWAdAy,1,170.0,0,18,0.0,1,0,3,0.0,0.0,1,3,1,1,1,0,2.0,0,445.0,0,585.0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,1,1,0,0,1,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0.000000,0
3,KPY-CA-6j8jNhqtKdh5,1,165.0,0,16,0.0,1,0,3,0.0,0.0,1,3,1,1,1,0,3.0,0,445.0,0,445.0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,1,1,0,0,1,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0.000000,0
4,KPY-CA-9Q6fbJPs3fC0,1,149.0,0,15,0.0,1,0,9,0.0,0.0,0,9,1,1,1,0,1.0,1,419.0,0,149.0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,1,1,0,0,1,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0.090909,0


In [5]:
test_df = pd.read_csv('data/fresh_mob_money_data_transformed.csv')

In [6]:
test_df.head()

,transaction_reference,merchant_bears_cost,amount,is_fraud,created_hod,phone_number_diff,unique_phone_numbers,diff_number_txns,total_transactions,proportion_different_numbers,average_phone_number_difference,shared_phone_number_flag,txn_count_per_customer,txn_count_per_customer_per_hour,max_txns_in_an_hour_per_customer,txn_count_per_customer_per_day,phone_txn_amount_suspicious,rolling_txn_count_per_customer,txn_spike_flag,avg_txn_amount_per_customer,unusual_txn_flag,rolling_mean_amount,amount_spike_flag,duplicate_txn_flag,odd_hour_flag,is_Exness,is_Nexasoft,is_Headway,is_Blenet,is_Hongkong,is_Bitolo,is_Onus,is_Astropay,is_Spoynt,is_Gateexpress,account_risk_level_above_average_risk,account_risk_level_high_risk,payment_status_expired,payment_status_failed,payment_status_success,payment_channel_api,payment_channel_modal,mobile_network_provider_Airtel,mobile_network_provider_MTN,mobile_network_provider_Orange,mobile_network_provider_Vodafone,is_success,is_insufficient_funds,is_charge_attempt_failed,is_wallet_not_active,is_invalid_recipient_wallet,is_invalid_PIN,created_dow_Friday,created_dow_Monday,created_dow_Saturday,created_dow_Sunday,created_dow_Thursday,created_dow_Tuesday,created_dow_Wednesday,confidence_score,temp_fraud
0,KPY-CA-EpgyBEXgO4M4,1,200.0,0,12,0.0,1,0,2,0.0,0.0,0,2,1,1,1,0,1.0,0,200.0,0,200.0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,1,1,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0.0,0
1,KPY-CA-ml54LusU74TP,1,200.0,0,9,0.0,1,0,2,0.0,0.0,0,2,1,1,1,0,2.0,0,200.0,0,200.0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,1,1,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0.0,0
2,KPY-CA-DEM93kyjKU1q,1,2928.0,0,16,0.0,1,0,1,0.0,0.0,0,1,1,1,1,0,1.0,0,2928.0,0,2928.0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,1,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0.0,0
3,KPY-CA-jvnSZ8J7B8X6,1,500.0,0,22,0.0,1,0,2,0.0,0.0,0,2,1,1,2,0,1.0,0,500.0,0,500.0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,1,1,0,1,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0.0,0
4,KPY-CA-Tn1mi7UqWfEy,1,500.0,0,18,0.0,1,0,2,0.0,0.0,0,2,1,1,2,0,2.0,0,500.0,0,500.0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,1,1,0,1,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0.0,0


In [7]:
test_df.drop(columns=['mobile_network_provider_Orange'], inplace=True)

In [4]:
X = df.drop(columns=['temp_fraud'], axis=1)
X.set_index('transaction_reference', inplace=True)
y = df['temp_fraud']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [6]:
import pickle
training_columns = X_train.columns.to_list()

with open("models/training_columns.pkl", "wb") as f:
    pickle.dump(training_columns, f)

In [7]:
X_train.columns

Index(['merchant_bears_cost', 'amount', 'is_fraud', 'created_hod',
       'phone_number_diff', 'unique_phone_numbers', 'diff_number_txns',
       'total_transactions', 'proportion_different_numbers',
       'average_phone_number_difference', 'shared_phone_number_flag',
       'txn_count_per_customer', 'txn_count_per_customer_per_hour',
       'max_txns_in_an_hour_per_customer', 'txn_count_per_customer_per_day',
       'phone_txn_amount_suspicious', 'rolling_txn_count_per_customer',
       'txn_spike_flag', 'avg_txn_amount_per_customer', 'unusual_txn_flag',
       'rolling_mean_amount', 'amount_spike_flag', 'duplicate_txn_flag',
       'odd_hour_flag', 'is_Exness', 'is_Nexasoft', 'is_Headway', 'is_Blenet',
       'is_Hongkong', 'is_Bitolo', 'is_Onus', 'is_Astropay', 'is_Spoynt',
       'is_Gateexpress', 'account_risk_level_above_average_risk',
       'account_risk_level_high_risk', 'payment_status_expired',
       'payment_status_failed', 'payment_status_success',
       'payment_channe

### logisitic regression (with sample weights)

In [9]:
sample_weights = X_train['confidence_score']

In [10]:
logmodel = LogisticRegression()
logmodel.fit(X_train, y_train, sample_weight=sample_weights) # add "sample_weight=sample_weights" after y_train

/opt/anaconda3/lib/python3.12/site-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

In [11]:
y_pred = logmodel.predict(X_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.92      0.16      0.27     47843
           1       0.43      0.98      0.60     31526

    accuracy                           0.48     79369
   macro avg       0.68      0.57      0.43     79369
weighted avg       0.73      0.48      0.40     79369



In [12]:
pd.DataFrame(confusion_matrix(y_test, y_pred), 
             index = ['Actual: False', 'Actual: True'],
             columns = ['Pred: False', 'Pred: True'])

,Pred: False,Pred: True
Actual: False,7440,40403
Actual: True,618,30908


### logisitic regression (without sample weights)

In [13]:
plain_logmodel = LogisticRegression()
plain_logmodel.fit(X_train, y_train) # add "sample_weight=sample_weights" after y_train

/opt/anaconda3/lib/python3.12/site-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

In [14]:
y_pred = plain_logmodel.predict(X_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.79      0.87      0.83     47843
           1       0.77      0.65      0.70     31526

    accuracy                           0.78     79369
   macro avg       0.78      0.76      0.77     79369
weighted avg       0.78      0.78      0.78     79369



In [15]:
pd.DataFrame(confusion_matrix(y_test, y_pred), 
             index = ['Actual: False', 'Actual: True'],
             columns = ['Pred: False', 'Pred: True'])

,Pred: False,Pred: True
Actual: False,41665,6178
Actual: True,11076,20450


### random forest

In [16]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
rf_model = RandomForestClassifier(random_state=42) # assign class_weights as 1:2 ratio
rf_model.fit(X_train, y_train)

RandomForestClassifier(random_state=42)

In [17]:
y_pred = rf_model.predict(X_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00     47843
           1       1.00      1.00      1.00     31526

    accuracy                           1.00     79369
   macro avg       1.00      1.00      1.00     79369
weighted avg       1.00      1.00      1.00     79369



In [18]:
pd.DataFrame(confusion_matrix(y_test, y_pred), 
             index = ['Actual: False', 'Actual: True'],
             columns = ['Pred: False', 'Pred: True'])

,Pred: False,Pred: True
Actual: False,47842,1
Actual: True,2,31524


### XGBoost

In [19]:
xgbmodel = xgb.XGBClassifier(
    objective='binary:logistic',  
    n_estimators=100,             # Number of boosting rounds
    learning_rate=0.1,            # Step size shrinkage
    max_depth=3,                  
    random_state=42,
    use_label_encoder=False,      
    eval_metric='logloss'          # Evaluation metric
)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
xgbmodel.fit(X_train, y_train)

/Users/christopher/.local/lib/python3.12/site-packages/xgboost/training.py:183: UserWarning: [15:51:19] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric='logloss',
              feature_types=None, feature_weights=None, gamma=None,
              grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.1, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=3, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=100, n_jobs=None,
              num_parallel_tree=None, ...)

In [20]:
y_pred = xgbmodel.predict(X_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00     47843
           1       1.00      1.00      1.00     31526

    accuracy                           1.00     79369
   macro avg       1.00      1.00      1.00     79369
weighted avg       1.00      1.00      1.00     79369



In [21]:
pd.DataFrame(confusion_matrix(y_test, y_pred), 
             index = ['Actual: False', 'Actual: True'],
             columns = ['Pred: False', 'Pred: True'])

,Pred: False,Pred: True
Actual: False,47843,0
Actual: True,0,31526


### iForest

In [57]:
from sklearn.ensemble import IsolationForest

In [59]:
iso_forest = IsolationForest(n_estimators=100, contamination=0.2)
iso_forest.fit(X_train)

IsolationForest(contamination=0.2)

In [61]:
# predicting anomalies (1 for inliers, -1 for outliers)
predictions = iso_forest.predict(X_train)

# calculates anomaly scores (lower scores indicate more anomalous)
scores = iso_forest.decision_function(X_train)

In [63]:
print("Predictions:", predictions)
print("Anomaly scores:", scores)

Predictions: [ 1  1  1 ... -1  1  1]
Anomaly scores: [ 0.11556207  0.07398128  0.08297943 ... -0.00430782  0.04880406
  0.05822262]


In [67]:
# Manually label some data as outliers for verification (replace with your ground truth)
manual_outlier_indices = [6,7,8] #indices of outliers in data array.
manual_outlier_labels = np.zeros(len(X_train))
manual_outlier_labels[manual_outlier_indices] = 1 #1 is outlier, 0 is inlier.

num_outliers = np.sum(predictions == -1)
print(f"Number of outliers identified: {num_outliers}")

Number of outliers identified: 63495


In [ ]:
# Convert Isolation Forest predictions to binary labels (1 for outliers, 0 for inliers)
binary_predictions = np.where(predictions == -1, 1, 0)

# Verify with actual data (if you have ground truth labels)
if np.any(manual_outlier_labels): #only run if manual labels exist.
    cm = confusion_matrix(manual_outlier_labels, binary_predictions)
    TN, FP, FN, TP = cm.ravel()

    print("\nConfusion Matrix:")
    print(cm)
    print(f"True Positives (TP): {TP}")
    print(f"True Negatives (TN): {TN}")
    print(f"False Positives (FP): {FP}")
    print(f"False Negatives (FN): {FN}")

    #Calculate additional metrics:
    accuracy = (TP+TN) / (TP+TN+FP+FN)
    precision = TP / (TP + FP) if (TP + FP) > 0 else 0
    recall = TP / (TP + FN) if (TP + FN) > 0 else 0
    f1 = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0

    print(f"Accuracy: {accuracy}")
    print(f"Precision: {precision}")
    print(f"Recall: {recall}")
    print(f"F1-score: {f1}")

else:
    print("\nManual outlier labels not provided. Cannot compute confusion matrix or metrics.")


In [69]:
iso_forest = IsolationForest(n_estimators=100, contamination=0.05, random_state=42) #Contamination matches the expected outlier ratio.
iso_forest.fit(X_train)

# Predict anomalies on X_test
predictions = iso_forest.predict(X_test)

# Count the number of outliers identified in X_test
num_outliers = np.sum(predictions == -1)
print(f"Number of outliers identified in X_test: {num_outliers}")

Number of outliers identified in X_test: 4061


In [71]:
# Convert Isolation Forest predictions to binary labels (1 for outliers, 0 for inliers)
binary_predictions = np.where(predictions == -1, 1, 0)

# Verify with actual y_test data
cm = confusion_matrix(y_test, binary_predictions)
TN, FP, FN, TP = cm.ravel()

print("\nConfusion Matrix:")
print(cm)
print(f"True Positives (TP): {TP}")
print(f"True Negatives (TN): {TN}")
print(f"False Positives (FP): {FP}")
print(f"False Negatives (FN): {FN}")


Confusion Matrix:
[[52177    68]
 [23131  3993]]
True Positives (TP): 3993
True Negatives (TN): 52177
False Positives (FP): 68
False Negatives (FN): 23131


In [73]:
# classification report:
accuracy = (TP+TN) / (TP+TN+FP+FN)
precision = TP / (TP + FP) if (TP + FP) > 0 else 0
recall = TP / (TP + FN) if (TP + FN) > 0 else 0
f1 = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0

print(f"Accuracy: {accuracy}")
print(f"Precision: {precision}")
print(f"Recall: {recall}")
print(f"F1-score: {f1}")

Accuracy: 0.70770703927226
Precision: 0.9832553558236887
Recall: 0.1472128004719068
F1-score: 0.2560846560846561


## Inference data

In [22]:
x_inf = test_df.drop(columns=['temp_fraud'], axis=1)
x_inf.set_index('transaction_reference', inplace=True)
y_inf = test_df['temp_fraud']

#### logistic regression (w/o sample weights)

In [23]:
y_pred = plain_logmodel.predict(x_inf)
print(classification_report(y_inf, y_pred))

              precision    recall  f1-score   support

           0       0.85      0.90      0.88    123117
           1       0.94      0.90      0.92    194276

    accuracy                           0.90    317393
   macro avg       0.89      0.90      0.90    317393
weighted avg       0.90      0.90      0.90    317393



In [24]:
pd.DataFrame(confusion_matrix(y_inf, y_pred), 
             index = ['Actual: False', 'Actual: True'],
             columns = ['Pred: False', 'Pred: True'])

,Pred: False,Pred: True
Actual: False,111047,12070
Actual: True,19475,174801


#### random forest

In [25]:
y_pred = rf_model.predict(x_inf)
print(classification_report(y_inf, y_pred))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00    123117
           1       1.00      1.00      1.00    194276

    accuracy                           1.00    317393
   macro avg       1.00      1.00      1.00    317393
weighted avg       1.00      1.00      1.00    317393



In [26]:
pd.DataFrame(confusion_matrix(y_inf, y_pred), 
             index = ['Actual: False', 'Actual: True'],
             columns = ['Pred: False', 'Pred: True'])

,Pred: False,Pred: True
Actual: False,123115,2
Actual: True,6,194270


#### xgboost

In [27]:
y_pred = xgbmodel.predict(x_inf)
print(classification_report(y_inf, y_pred))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00    123117
           1       1.00      1.00      1.00    194276

    accuracy                           1.00    317393
   macro avg       1.00      1.00      1.00    317393
weighted avg       1.00      1.00      1.00    317393



In [28]:
pd.DataFrame(confusion_matrix(y_inf, y_pred), 
             index = ['Actual: False', 'Actual: True'],
             columns = ['Pred: False', 'Pred: True'])

,Pred: False,Pred: True
Actual: False,123115,2
Actual: True,0,194276


In [29]:
import pickle

In [31]:
filename1 = "models/ghs_fraud_model_logreg.pkl"
filename2 = "models/ghs_fraud_model_rfc.pkl"
filename3 = "models/ghs_fraud_model_xgb.pkl"

In [32]:
try:
    with open(filename1, 'wb') as file:
        pickle.dump(plain_logmodel, file)
    print(f"Model successfully pickled and saved as '{filename1}'")
except Exception as e:
    print(f"An error occurred during pickling: {e}")

Model successfully pickled and saved as 'models/ghs_fraud_model_logreg.pkl'


In [33]:
try:
    with open(filename2, 'wb') as file:
        pickle.dump(rf_model, file)
    print(f"Model successfully pickled and saved as '{filename2}'")
except Exception as e:
    print(f"An error occurred during pickling: {e}")

Model successfully pickled and saved as 'models/ghs_fraud_model_rfc.pkl'


In [34]:
try:
    with open(filename3, 'wb') as file:
        pickle.dump(xgbmodel, file)
    print(f"Model successfully pickled and saved as '{filename3}'")
except Exception as e:
    print(f"An error occurred during pickling: {e}")

Model successfully pickled and saved as 'models/ghs_fraud_model_xgb.pkl'
